Analyze web-hosted JSON data
============================

This notebook reads and processes JSON-encoded data hosted on the web using a combination of [Dask Bag](https://docs.dask.org/en/latest/bag.html) and [Dask Dataframe](https://docs.dask.org/en/latest/dataframe.html).

This data comes from [mybinder.org](https://mybinder.org) a web service to run Jupyter notebooks live on the web (you may be running this notebook there now).  My Binder publishes records for every time someone launches a live notebook like this one, and stores that record in a publicly accessible JSON file, one file per day. 

## Introduction to the dataset

This data is stored as JSON-encoded text files on the public web.  Here are some example lines.

In [14]:
import dask.bag as db
db.read_text('https://archive.analytics.mybinder.org/events-2018-11-03.jsonl').take(3)

('{"timestamp": "2018-11-03T00:00:00+00:00", "schema": "binderhub.jupyter.org/launch", "version": 1, "provider": "GitHub", "spec": "Qiskit/qiskit-tutorial/master", "status": "success"}\n',
 '{"timestamp": "2018-11-03T00:00:00+00:00", "schema": "binderhub.jupyter.org/launch", "version": 1, "provider": "GitHub", "spec": "ipython/ipython-in-depth/master", "status": "success"}\n',
 '{"timestamp": "2018-11-03T00:00:00+00:00", "schema": "binderhub.jupyter.org/launch", "version": 1, "provider": "GitHub", "spec": "QISKit/qiskit-tutorial/master", "status": "success"}\n')

We see that it includes one line for every time someone started a live notebook on the site.  It includes the time that the notebook was started, as well as the repository from which it was served.

In this notebook we'll look at many such files, parse them from JSON to Python dictionaries, and then from there to Pandas dataframes.  We'll then do some simple analyses on this data.

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will start the dashboard which 
is useful to gain insight on the computation.  

In [1]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=1, 
                n_workers=4,
                memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:40417 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.00 GB


## Get a list of files on the web

The mybinder.org team maintains an index file that points to all other available JSON files of data.  Lets convert this to a list of URLs that we'll read in the next section.

In [2]:
import dask.bag as db
import json

In [3]:
db.read_text('https://archive.analytics.mybinder.org/index.jsonl').map(json.loads).compute()

[{'name': 'events-2018-11-03.jsonl', 'date': '2018-11-03', 'count': '7057'},
 {'name': 'events-2018-11-04.jsonl', 'date': '2018-11-04', 'count': '7489'},
 {'name': 'events-2018-11-05.jsonl', 'date': '2018-11-05', 'count': '13590'},
 {'name': 'events-2018-11-06.jsonl', 'date': '2018-11-06', 'count': '13920'},
 {'name': 'events-2018-11-07.jsonl', 'date': '2018-11-07', 'count': '12766'},
 {'name': 'events-2018-11-08.jsonl', 'date': '2018-11-08', 'count': '13666'}]

In [4]:
filenames = (db.read_text('https://archive.analytics.mybinder.org/index.jsonl')
               .map(json.loads)
               .pluck('name')
               .compute())

filenames = ['https://archive.analytics.mybinder.org/' + fn for fn in filenames]
filenames[:5]

['https://archive.analytics.mybinder.org/events-2018-11-03.jsonl',
 'https://archive.analytics.mybinder.org/events-2018-11-04.jsonl',
 'https://archive.analytics.mybinder.org/events-2018-11-05.jsonl',
 'https://archive.analytics.mybinder.org/events-2018-11-06.jsonl',
 'https://archive.analytics.mybinder.org/events-2018-11-07.jsonl']

## Create Bag of all events

We now create a [Dask Bag](https://docs.dask.org/en/latest/bag.html) around that list of URLs, and then call the `json.loads` function on every line to turn those lines of JSON-encoded text into Python dictionaries that can be more easily manipulated.

In [5]:
events = db.read_text(filenames).map(json.loads)
events.take(2)

({'timestamp': '2018-11-03T00:00:00+00:00',
  'schema': 'binderhub.jupyter.org/launch',
  'version': 1,
  'provider': 'GitHub',
  'spec': 'Qiskit/qiskit-tutorial/master',
  'status': 'success'},
 {'timestamp': '2018-11-03T00:00:00+00:00',
  'schema': 'binderhub.jupyter.org/launch',
  'version': 1,
  'provider': 'GitHub',
  'spec': 'ipython/ipython-in-depth/master',
  'status': 'success'})

## Most Popular Binders

Lets do a simple frequency count to find those binders that are run the most often.

In [6]:
events.pluck('spec').frequencies(sort=True).take(20)

(('ipython/ipython-in-depth/master', 32270),
 ('jupyterlab/jupyterlab-demo/master', 8013),
 ('ines/spacy-io-binder/live', 2798),
 ('binder-examples/r/master', 1765),
 ('bokeh/bokeh-notebooks/master', 1640),
 ('DS-100/textbook/master', 1239),
 ('mkozturk/CMPE140/master', 1037),
 ('rationalmatter/juno-demo-notebooks/master', 994),
 ('QuantStack/xeus-cling/stable', 977),
 ('marlohaering/SE1-Notebook/master', 841),
 ('RasaHQ/rasa_core/master', 635),
 ('binder-examples/julia-python/master', 555),
 ('QISKit/qiskit-tutorial/master', 554),
 ('quasiben/fiftyfizzbuzzes/master', 536),
 ('numba/numba-examples/master', 462),
 ('geoscixyz/gpgLabs/master', 370),
 ('nteract/examples/master', 347),
 ('dask/dask-examples/master', 347),
 ('data-8/textbook/gh-pages', 333),
 ('SoftmatterLMU-RaedlerGroup/notebooks_e3_ws1819/master', 329))

## Convert to Dask Dataframe

Finally, we can convert our bag of Python dictionaries into a [Dask Dataframe](https://docs.dask.org/en/latest/dataframe.html), and follow up with more Pandas-like computations.

We'll do the same computation as above, now with Pandas syntax.

In [7]:
df = events.to_dataframe()
df.head()

,provider,schema,spec,status,timestamp,version
0,GitHub,binderhub.jupyter.org/launch,Qiskit/qiskit-tutorial/master,success,2018-11-03T00:00:00+00:00,1
1,GitHub,binderhub.jupyter.org/launch,ipython/ipython-in-depth/master,success,2018-11-03T00:00:00+00:00,1
2,GitHub,binderhub.jupyter.org/launch,QISKit/qiskit-tutorial/master,success,2018-11-03T00:00:00+00:00,1
3,GitHub,binderhub.jupyter.org/launch,QISKit/qiskit-tutorial/master,success,2018-11-03T00:01:00+00:00,1
4,GitHub,binderhub.jupyter.org/launch,jupyterlab/jupyterlab-demo/master,success,2018-11-03T00:01:00+00:00,1


In [15]:
df.spec.value_counts().nlargest(20).to_frame().compute()

,spec
ipython/ipython-in-depth/master,32270
jupyterlab/jupyterlab-demo/master,8013
ines/spacy-io-binder/live,2798
binder-examples/r/master,1765
bokeh/bokeh-notebooks/master,1640
DS-100/textbook/master,1239
mkozturk/CMPE140/master,1037
rationalmatter/juno-demo-notebooks/master,994
QuantStack/xeus-cling/stable,977
marlohaering/SE1-Notebook/master,841


## Persist in memory

This dataset fits nicely into memory. Lets avoid pulling from the cloud every time we do an operation and instead keep data local.

In [8]:
df = df.persist()

Honestly, at this point it makes more sense to just switch to Pandas, but this is a Dask example, so we'll continue with Dask dataframe.

## Investigate providers other than Github

Most binders are specified as git repositories on GitHub, but not all.  Lets investigate other providers.

In [9]:
import urllib

In [10]:
df.provider.value_counts().compute()

GitHub    68126
GitLab      282
Git          80
Name: provider, dtype: int64

In [11]:
(df[df.provider == 'GitLab']
 .spec
 .map(urllib.parse.unquote, meta=('spec', object))
 .value_counts()
 .to_frame()
 .compute())

,spec
rruizz/inforfis/master,61
dbernhard/JavaM2/master,61
dbernhard/PythonM1/master,38
PersonalDataIO/toronto-letter/master,37
clemej/data601-clemens-fall18/master,24
wichit2s/programmingfundamentals/master,13
tbalem/sml/master,12
jumson/sdr-lab-machines/master,7
PersonalDataIO/gdpr-rights-notebook/master,6
PersonalDataIO/uber-data/testing,5


In [12]:
(df[df.provider == 'Git']
 .spec
 .apply(urllib.parse.unquote, meta=('spec', object))
 .value_counts()
 .to_frame()
 .compute())

,spec
https://bitbucket.org/gaur/1820/a4027afe0aa592d49f3989b2e9c8136c36322a77,31
https://collaborating.tuhh.de/cip3725/ib_base.git/0a1f4f66a1a3c29ff347b2abc79bb292b0be17ca,17
https://gitlab.oceantrack.org/otndc/acoustic-telemetry-analysis-workshop/ed80a18d3ba7e4a39716ef7255cc954ce022710f,7
https://gricad-gitlab.univ-grenoble-alpes.fr/nonsmooth/siconos-tutorial.git/b08a0514b22b3927b58bddce3c4018f27ac0fc7d,6
https://github.com/HAMID-ALI/Python3_programs/6b1af958ac574f9be5d43504e161a57c351425ea,4
https://gaur@bitbucket.org/gaur/1820.git/b455e5db47da15dc468083d830a08b7e460abbcd,4
https://opensource.ncsa.illinois.edu/bitbucket/scm/smm/smm-network-analysis.git/44acd332da1571acb412db803991e5f1dfc7a329,3
https://gitlab.mech.kuleuven.be/rob-expressiongraphs/docker/etasl-binder.git/127d6c9f33a938a98607505ef30237b5223000e4,3
https://opensource.ncsa.illinois.edu/bitbucket/scm/smm/smm-network-analysis.git/fdc417158238cfabed8ddec6e9a1de36eb2e469f,1
https://gricad-gitlab.univ-grenoble-alpes.fr/nonsmooth/siconos-tutorials.git/b7ce70e869084ff0c2f93c1be34a9dda8a513c44,1
